### pseudocode for calculator
* define raw resources / node type 
* usable = raw-overhead
* define desired hardware tier
* total resources per execution = (hardware tier def + execution overhead) 
* max executions = int(min(usable cpu/total cpu per execution, usable mem/total mem per execution))
* define overhead for cpu, memory
* produce overhead metrics
* define idle cpu, memory
* produce utilization metrics

In [15]:
import pandas as pd

In [16]:
import sys
import os

In [17]:
df = pd.read_csv('ec2_ref.csv')

In [18]:
df.head()

,Instance,vCPU*,Mem (GiB),Storage,Dedicated EBS Bandwidth (Mbps),Network Performance
0,m5.xlarge,4,16.0,EBS-Only,Up to 10,"Up to 4,750"
1,m5.2xlarge,8,32.0,EBS-Only,Up to 10,"Up to 4,750"
2,m5.4xlarge,16,64.0,EBS-Only,Up to 10,"4,750"
3,m5.8xlarge,32,128.0,EBS Only,10,"6,800"
4,m5.12xlarge,48,192.0,EBS-Only,10,"9,500"


In [19]:
hw_tier_catalogue = df[['Instance','vCPU*','Mem (GiB)']]
hw_tier_catalogue.head()

,Instance,vCPU*,Mem (GiB)
0,m5.xlarge,4,16.0
1,m5.2xlarge,8,32.0
2,m5.4xlarge,16,64.0
3,m5.8xlarge,32,128.0
4,m5.12xlarge,48,192.0


In [20]:
len(hw_tier_catalogue)

20

In [21]:
for i in range(1):
    print(hw_tier_catalogue.iloc[i,:].tolist())
    cpu = hw_tier_catalogue.iloc[i,1]
    mem = hw_tier_catalogue.iloc[i,2]
    print('usable cpu: '+str(cpu), cpu)
    print('usable mem: '+str(mem), mem)

['m5.xlarge', 4, 16.0]
usable cpu: 4 4
usable mem: 16.0 16.0


In [22]:
hw_tier_catalogue['vCPU*'][0]

4

In [23]:
hw_tier_catalogue.iloc[0][0]

'm5.xlarge'

In [24]:
global node_overhead_cpu
global node_overhead_mem
global exec_overhead_cpu
global exec_overhead_mem

node_overhead_cpu = 1.5 #cores
node_overhead_mem = 2  #GiB

exec_overhead_cpu = 1 #cores
exec_overhead_mem = 1 #GiB

In [28]:
def reserved_cpu(cpu):
    if cpu <=1.0:
        #6% of the first core
        reserved = .06*cpu
    elif (cpu >1.0 and cpu<=2.0):
        #1% of the next core (up to 2 cores)
        reserved = .06*1.0 + .01*(cpu-1.0)
    elif (cpu >2.0 and cpu <=4.0):
        #0.5% of the next 2 cores (up to 4 cores)
        reserved = .06*1.0 + .01*(1.0) + .005*(cpu-2.0)
    else: 
        reserved = .06*(1.0) + .01*(1.0) + .005*(2.0) + .0025*(cpu-4)
    
    node_overhead_cpu = reserved
    return node_overhead_cpu

In [29]:
reserved_cpu(1.0)

0.06

In [30]:
reserved_cpu(2.0)

0.06999999999999999

In [31]:
reserved_cpu(4.0)

0.07999999999999999

In [32]:
reserved_cpu(5.0)

0.08249999999999999

In [33]:
reserved_cpu(8.0)

0.08999999999999998

In [46]:
def reserved_mem(mem):
    
    kubelet_eviction_reserve = .100
    
    if mem <=1.0:
        #255 MiB of memory for machines with less than 1 GB of memory
        reserved = 0.255
    elif (mem >1.0 and mem<=4.0):
        #25% of the first 4GB of memory
        reserved = .25*mem
    elif (mem >4.0 and mem <=8.0):
        #20% of the next 4GB of memory (up to 8GB)
        reserved = .25*(4.0) + .20*(mem-4.0)
    elif (mem >8.0 and mem <=16.0):
        #10% of the next 8GB of memory (up to 16GB)
        reserved = .25*(4.0) + .20*(4.0) + .10*(mem-8.0)
    elif (mem >16.0 and mem <=128.0):
        #6% of the next 112GB of memory (up to 128GB)
        reserved = .25*(4.0) + .20*(4.0) + .10*(8.0) + .06*(mem-16.0)
    else: 
        #2% of any memory above 128GB
        reserved = .25*(4.0) + .20*(4.0) + .10*(8.0) + .06*(112.0) + .02*(mem-128.0)
    
    node_overhead_mem = reserved + kubelet_eviction_reserve
    return node_overhead_mem

In [47]:
reserved_mem(1.0)

0.355

In [48]:
reserved_mem(4.0)

1.1

In [49]:
reserved_mem(8.0)

1.9000000000000001

In [50]:
reserved_mem(16.0)

2.7

In [51]:
reserved_mem(128.0)

9.42

In [53]:
reserved_mem(256.0)

11.98

In [55]:
.25*4.0 + .20*4.0 + .10*8.0 + .06*112.0 + .02*128 + 0.1

11.98

In [186]:
def max_exec(hw_cpu, hw_mem, exec_cpu, exec_mem):
    
    usable_cpu = hw_cpu - node_overhead_cpu
    #print(usable_cpu)
    usable_mem = hw_mem - node_overhead_mem
    #print(usable_mem)
    
    run_cpu = exec_cpu + exec_overhead_cpu
    #print(run_cpu)
    run_mem = exec_mem + exec_overhead_mem
    #print(run_mem)
    
    execs = 0
    
    while (usable_cpu >= run_cpu and usable_mem >= run_mem): 
        
        execs += 1

        usable_cpu -= run_cpu
        usable_mem -= run_mem
   
    remaining_cpu = usable_cpu
    remaining_mem = usable_mem
    
    o_cpu = node_overhead_cpu + (execs*exec_overhead_cpu)
    o_mem = node_overhead_mem + (execs*exec_overhead_mem)
    
    print('execs: '+str(execs))
    print('remaining_cpu: '+str(remaining_cpu))
    print('remaining_mem: '+str(remaining_mem))
    print('total_cpu_overhead: '+str(o_cpu))
    print('total_mem_overhead: '+str(o_mem))
    
    return execs, remaining_cpu, remaining_mem, o_cpu, o_mem

In [203]:
exec_cpu = 2
exec_mem = 8

In [215]:
execs, remaining_cpu, remaining_mem, o_cpu, o_mem = max_exec(36, 72, exec_cpu, exec_mem)

execs: 7
remaining_cpu: 13.5
remaining_mem: 7
total_cpu_overhead: 8.5
total_mem_overhead: 9


In [216]:
def overhead(execs, exec_cpu, exec_mem):

    if execs !=0:
        o_cpu = node_overhead_cpu + (execs*exec_overhead_cpu)
        o_mem = node_overhead_mem + (execs*exec_overhead_mem)
        
        print('total_cpu_overhead: '+str(o_cpu))

        tot_cpu = execs * exec_cpu
        print('total execution cores: '+str(tot_cpu))

        tot_mem = execs * exec_mem

        #cpu overhead / total_exec_corese
        ratio_cpu = o_cpu/tot_cpu
        print('overhead ratio, cpu: '+str(ratio_cpu))
        ratio_mem = o_mem/tot_mem
        
    else: #temp set to 100% overhead
        ratio_cpu = 1
        ratio_mem = 1
    
    return ratio_cpu, ratio_mem

In [217]:
overhead(7, 2, 8)

total_cpu_overhead: 8.5
total execution cores: 14
overhead ratio, cpu: 0.6071428571428571


(0.6071428571428571, 0.16071428571428573)

In [228]:
def utilization(hw_cpu, hw_mem, remaining_cpu, remaining_mem):
    #utilization = (exec_resource_def - idle _resource) / exec_resource_def
    util_cpu = (hw_cpu - remaining_cpu)/hw_cpu
    util_mem = (hw_mem - remaining_mem)/hw_mem
    
    return util_cpu, util_mem

In [219]:
utilization(36,72)

(0.625, 0.9027777777777778)

In [256]:
def node_sizing(requested_cpu, requested_mem, hw_tier_data):
    
    df_results = pd.DataFrame(columns=['node_size', 'max_executions', 'overhead_ratio_cpu', 'overhead_ratio_mem', 
                                   'util_ratio_cpu', 'util_ratio_mem', 'efficiency_metric'])
    
    for i in range(len(hw_tier_data)):
        hw_cpu = hw_tier_data.iloc[i,1]
        #print(hw_cpu)
        hw_mem = hw_tier_data.iloc[i,2]
        #print(hw_mem)
        
        usable_cpu = hw_cpu - node_overhead_cpu
        usable_mem = hw_mem - node_overhead_mem
        
        if (usable_cpu > requested_cpu and usable_mem > requested_mem):
                        
            print('core count for eval: '+str(hw_cpu))
            print('memory for eval (GB): '+str(hw_mem))
            
            execs, remaining_cpu, remaining_mem, o_cpu, o_mem = max_exec(hw_cpu, hw_mem, requested_cpu, requested_mem)
        
            overhead_cpu, overhead_mem = overhead(execs, requested_cpu, requested_mem)
        
            util_cpu, util_mem = utilization(hw_cpu, hw_mem, remaining_cpu, remaining_mem)
        
            eff_cpu = util_cpu/overhead_cpu
            eff_mem = util_mem/overhead_mem
        
            efficiency_metric = eff_cpu + eff_mem
        
            df_result = pd.DataFrame({'node_size':hw_tier_data.iloc[i,0],
                                      'max_executions':execs,
                                      'overhead_ratio_cpu': overhead_cpu,
                                      'overhead_ratio_mem': overhead_mem,
                                      'util_ratio_cpu': util_cpu,
                                      'util_ratio_mem': util_mem,
                                      'efficiency_metric': efficiency_metric}, index = [0])
        
            #print(df_result.shape)
        
            df_results = df_results.append(df_result, ignore_index = True)
            
    #print(df_results.shape)
        
    return df_results

In [257]:
requested_cpu = 2
requested_mem = 8

In [258]:
results = node_sizing(requested_cpu, requested_mem, df)

core count for eval: 4
memory for eval (GB): 16.0
execs: 0
remaining_cpu: 2.5
remaining_mem: 14.0
total_cpu_overhead: 1.5
total_mem_overhead: 2
core count for eval: 8
memory for eval (GB): 32.0
execs: 2
remaining_cpu: 0.5
remaining_mem: 12.0
total_cpu_overhead: 3.5
total_mem_overhead: 4
total_cpu_overhead: 3.5
total execution cores: 4
overhead ratio, cpu: 0.875
core count for eval: 16
memory for eval (GB): 64.0
execs: 4
remaining_cpu: 2.5
remaining_mem: 26.0
total_cpu_overhead: 5.5
total_mem_overhead: 6
total_cpu_overhead: 5.5
total execution cores: 8
overhead ratio, cpu: 0.6875
core count for eval: 32
memory for eval (GB): 128.0
execs: 10
remaining_cpu: 0.5
remaining_mem: 36.0
total_cpu_overhead: 11.5
total_mem_overhead: 12
total_cpu_overhead: 11.5
total execution cores: 20
overhead ratio, cpu: 0.575
core count for eval: 48
memory for eval (GB): 192.0
execs: 15
remaining_cpu: 1.5
remaining_mem: 55.0
total_cpu_overhead: 16.5
total_mem_overhead: 17
total_cpu_overhead: 16.5
total executi

In [259]:
results = results.sort_values(['max_executions', 'efficiency_metric'], ascending=[False, False]).reset_index(drop=True)

In [260]:
results

,node_size,max_executions,overhead_ratio_cpu,overhead_ratio_mem,util_ratio_cpu,util_ratio_mem,efficiency_metric
0,m5.24xlarge,31,0.524194,0.133065,0.984375,0.731771,7.377253
1,c5.24xlarge,21,0.535714,0.136905,0.671875,0.994792,8.520471
2,m5.16xlarge,20,0.5375,0.1375,0.960938,0.710938,6.958245
3,c5.18xlarge,15,0.55,0.141667,0.645833,0.951389,7.889929
4,m5.12xlarge,15,0.55,0.141667,0.968750,0.713542,6.798128
5,c5.12xlarge,10,0.575,0.15,0.656250,0.958333,7.530193
6,m5.8xlarge,10,0.575,0.15,0.984375,0.718750,6.503623
7,c5.9xlarge,7,0.607143,0.160714,0.625000,0.902778,6.646696
8,c4.8xlarge,6,0.625,0.166667,0.541667,0.933333,6.466667
9,m5.4xlarge,4,0.6875,0.1875,0.843750,0.593750,4.393939


In [261]:
results.to_html(os.environ['DOMINO_WORKING_DIR']+'/results/node_recommendations_{}_{}.html'.format(requested_cpu, requested_mem))